### Reference

1. https://github.com/ashishpatel26/LLM-Finetuning/blob/main/7.FineTune_LLAMA2_with_QLORA.ipynb  
2. https://huggingface.co/blog/4bit-transformers-bitsandbytes  
3. https://pytorch.org/blog/finetune-llms/  
4. https://colab.research.google.com/drive/1vIjBtePIZwUaHWfjfNHzBjwuXOyU_ugD?usp=sharing#scrollTo=6k_nL6xJMZW2  

# 1. Weight and Bias Login

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune_06"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

# 2. Login Huggingface

In [2]:
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 3. Dataset Load

In [3]:
from datasets import load_dataset

In [4]:
ko_instruction_01 = load_dataset("nlpai-lab/kullm-v2", cache_dir="/mnt/t7/.cache/huggingface/datasets", split="train")

In [5]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=2160)

In [6]:
ko_instruction_01

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})

In [7]:
ko_instruction_01 = ko_instruction_01.train_test_split(test_size=0.92)

In [8]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['id', 'instruction', 'input', 'output'],
        num_rows: 12210
    })
    test: Dataset({
        features: ['id', 'instruction', 'input', 'output'],
        num_rows: 140420
    })
})

In [9]:
ko_instruction_01["train"][100]

{'id': 'vicuna_{idx}',
 'instruction': '일상적인 활동에서 애자일과 린을 사용하는 것에 대한 성명서를 작성하시겠습니까?',
 'input': '',
 'output': '"[회사 이름]에서는 고객에게 최상의 결과를 제공하기 위해 프로세스를 지속적으로 개선하고 간소화해야 한다고 믿습니다. 그렇기 때문에 애자일 및 린 방법론을 일상적인 활동에 통합합니다. 민첩성을 수용함으로써 변화하는 요구사항에 빠르게 적응하고 가장 중요한 업무의 우선순위를 정할 수 있습니다. 또한 린의 원칙을 통해 낭비를 없애고 고객에게 최대한 효율적으로 가치를 제공하는 데 집중합니다. 이러한 접근 방식을 결합하여 고객의 요구를 충족하는 고품질의 제품과 서비스를 제공하는 동시에 내부 프로세스를 개선할 수 있습니다. 이를 통해 경쟁에서 앞서 나가고 강력한 경쟁 우위를 유지할 수 있습니다."'}

# 4. Loading the Model

In [10]:
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}
cache_model_dir="/mnt/t7/.cache/huggingface/models"

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [12]:
# Settings for 4-bit QLoRA Training(4bit QLoRA 학습을 위한 설정)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_compute_dtype=torch.bfloat16, # Nvidia의 Ampere 아키텍처 이후 가속기는 bf16으로 속도 향상을 꾀할수 있다. 
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# bnb_4bit_quant_type="nf4" 설정상 기본값은 bnb_4bit_quant_type="fp4"이나 허깅페이스 저자들에 의하면
# 경험적 결과로 "nf4"가 결과가 더 좋았다고 한다. https://huggingface.co/blog/4bit-transformers-bitsandbytes
# bnb_4bit_use_double_quant=True로 하면 매개변수단 0.4bit을 추가로 절약 할 수 있다고 한다. 

In [13]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map=device_map, cache_dir=cache_model_dir, trust_remote_code=True)
model.config.use_cache = False

# model.config.pretraining_tp = 1
# 종종 QLoRA 코드에 이 코드가 보이는데 병렬 학습에 쓰이는 코드로 보인다. 

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, cache_dir=cache_model_dir)
tokenizer.add_special_tokens({'pad_token': '<PAD>'})



1

In [15]:
# 이 코드를 쓰지 않는 경우(물론 패딩 토큰을 별도로 사용하는 경우에 해당됨) loss가 0으로 떨어지는 경우가 있다함
tokenizer.padding_side = "right"

In [16]:
model.resize_token_embeddings(len(tokenizer)) # pad_token이 추가되었으므로 embedding과 language modeling head를 resize

Embedding(46337, 4096)

# 5. LoRA Setup

In [17]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

In [18]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

# 6. Formatting Dataset

In [19]:
def format_instruction(sample):
    system_prompt = f"### instruction: {sample['instruction']}"
    input = f"### input: {sample['input']}" if len(sample["input"]) > 0 else None
    output = f"### output: {sample['output']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}{tokenizer.eos_token}"
    return sample

In [20]:
train_dataset = ko_instruction_01['train'].map(template_dataset, remove_columns=list(ko_instruction_01['train'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/12210 [00:00<?, ? examples/s]

In [21]:
train_dataset["text"][100]

'### instruction: 일상적인 활동에서 애자일과 린을 사용하는 것에 대한 성명서를 작성하시겠습니까?\n\n### output: "[회사 이름]에서는 고객에게 최상의 결과를 제공하기 위해 프로세스를 지속적으로 개선하고 간소화해야 한다고 믿습니다. 그렇기 때문에 애자일 및 린 방법론을 일상적인 활동에 통합합니다. 민첩성을 수용함으로써 변화하는 요구사항에 빠르게 적응하고 가장 중요한 업무의 우선순위를 정할 수 있습니다. 또한 린의 원칙을 통해 낭비를 없애고 고객에게 최대한 효율적으로 가치를 제공하는 데 집중합니다. 이러한 접근 방식을 결합하여 고객의 요구를 충족하는 고품질의 제품과 서비스를 제공하는 동시에 내부 프로세스를 개선할 수 있습니다. 이를 통해 경쟁에서 앞서 나가고 강력한 경쟁 우위를 유지할 수 있습니다."</s>'

# 7. Training Argument Setup

In [22]:
from transformers import TrainingArguments

In [ ]:
checkpoint_dir = "./checkpoint/experi_08"

In [23]:
output_dir = checkpoint_dir
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
report_to="wandb"
save_steps = 20
save_total_limit=5
num_train_epochs = 2
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [24]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    logging_steps=logging_steps,
    report_to = report_to,
    learning_rate=learning_rate,
    bf16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [25]:
from trl import SFTTrainer

# max_seq_length는 필수는 아니다. 오히려 지정했을 경우 인퍼때 훈련때 지정한 길이보다 넘어서서 설정하면 오류가 나는 듯 하다.
max_seq_length = 1024

In [26]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/12210 [00:00<?, ? examples/s]

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [27]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# 8. Training

In [28]:
trainer.train()

  0%|          | 0/12210 [00:00<?, ?it/s]

{'loss': 2.5156, 'grad_norm': 0.10693359375, 'learning_rate': 5.449591280653951e-06, 'epoch': 0.0}
{'loss': 2.5216, 'grad_norm': 0.1396484375, 'learning_rate': 1.0899182561307902e-05, 'epoch': 0.0}


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 2.866, 'grad_norm': 0.17578125, 'learning_rate': 1.6348773841961854e-05, 'epoch': 0.0}
{'loss': 2.8378, 'grad_norm': 0.2236328125, 'learning_rate': 2.1798365122615804e-05, 'epoch': 0.01}


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


...

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 1.7115, 'grad_norm': 0.4765625, 'learning_rate': 3.377522587182302e-07, 'epoch': 2.0}
{'loss': 1.7684, 'grad_norm': 0.93359375, 'learning_rate': 1.688761293591151e-07, 'epoch': 2.0}


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 1.466, 'grad_norm': 0.671875, 'learning_rate': 0.0, 'epoch': 2.0}
{'train_runtime': 15517.1967, 'train_samples_per_second': 1.574, 'train_steps_per_second': 0.787, 'train_loss': 1.6900457468118752, 'epoch': 2.0}


TrainOutput(global_step=12210, training_loss=1.6900457468118752, metrics={'train_runtime': 15517.1967, 'train_samples_per_second': 1.574, 'train_steps_per_second': 0.787, 'train_loss': 1.6900457468118752, 'epoch': 2.0})

In [ ]:
lora_model_save_dir = "./results/experi_08"

In [29]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(lora_model_save_dir, save_embedding_layers = True)

In [30]:
lora_config = LoraConfig.from_pretrained(lora_model_save_dir)
model = get_peft_model(model, lora_config)

In [35]:
tokenizer.push_to_hub('aeolian83/llama_ko_sft_kullm_experi_05')

CommitInfo(commit_url='https://huggingface.co/aeolian83/llama_ko_sft_kullm_experi_05/commit/a660bf40db1d86c7dea85c0a2328f73c61f61596', commit_message='Upload tokenizer', commit_description='', oid='a660bf40db1d86c7dea85c0a2328f73c61f61596', pr_url=None, pr_revision=None, pr_num=None)